# CCSD theory for a closed-shell reference

In this notebook we will use wicked to generate and implement equations for the CCSD method.
To simplify this notebook some of the utility functions are imported from the file `examples_helpers.py`.
In this example, we run a CCSD computation on the H<sub>6</sub> molecule, reading all the relevant information from the file `sr-h2o-cc-pvdz-spinorbital.npy`.

In [18]:
import time
import wicked as w
import numpy as np
from examples_helpers import *
import copy

## Read calculation information (integrals, number of orbitals)

We start by reading information about the reference state, integrals, and denominators from the file `sr-h2o-cc-pvdz-spinorbital.npy`. The variable `H` is a dictionary that holds the blocks of the Hamiltonian **normal-ordered** with respect to the Hartree–Fock determinant. `invD` similarly is a dictionary that stores the denominators $(\epsilon_i + \epsilon_j + \ldots - \epsilon_a - \epsilon_b - \ldots)^{-1}$.

In [19]:
molecule = "sr-h2o-sto-3g-spinorbital"

with open(f"{molecule}.npy", "rb") as f:
    Eref = np.load(f)
    Ecorr_ref = np.load(f)
    nocc, nvir = np.load(f)
    H = np.load(f, allow_pickle=True).item()

invD = compute_inverse_denominators(H, nocc, nvir, 2)

## Compute the MP2 energy

To verify that the Hamiltonian is read correctly, we compute the MP2 correlation energy

In [20]:
# Compute the MP2 correlation energy
Emp2 = 0.0
for i in range(nocc):
    for j in range(nocc):
        for a in range(nvir):
            for b in range(nvir):
                Emp2 += 0.25 * H["oovv"][i][j][a][b] ** 2 * invD["oovv"][i][j][a][b]

print(f"MP2 correlation energy: {Emp2:.12f} Eh")

MP2 correlation energy: -0.049149644668 Eh


## Define orbital spaces and the Hamiltonian and cluster operators

Here we define the cluster operator (`Top`) and the Hamiltonian (`Hop`) that will be used to derive the CCSD equations. We also define the similarity-transformed Hamiltonian $\bar{H}$ truncated at the four-nested commutator:

\begin{equation}
\bar{H} = \hat{H} + [\hat{H},\hat{T}] + \frac{1}{2} [[\hat{H},\hat{T}],\hat{T}]
+ \frac{1}{6} [[[\hat{H},\hat{T}],\hat{T}],\hat{T}]
+ \frac{1}{24} [[[[\hat{H},\hat{T}],\hat{T}],\hat{T}],\hat{T}] + \ldots
\end{equation}

In [21]:
w.reset_space()
w.add_space("o", "fermion", "occupied", ["i", "j", "k", "l", "m", "n"])
w.add_space("v", "fermion", "unoccupied", ["a", "b", "c", "d", "e", "f"])

Top = w.op("T", ["v+ o", "v+ v+ o o"], unique=False)
Hop = w.utils.gen_op("H", 1, "ov", "ov") + w.utils.gen_op("H", 2, "ov", "ov")
# the similarity-transformed Hamiltonian truncated to the four-nested commutator term
Hbar = w.bch_series(Hop, Top, 4)

In the following lines, we apply Wick's theorem to simplify the similarity-transformed Hamiltonian $\bar{H}$ computing all contributions ranging from operator rank 0 to 4 (double substitutions).
Then we convert all the terms into many-body equations accumulated into the residual `R`.

In [23]:
wt = w.WickTheorem()
expr = wt.contract(w.rational(1), Hbar, 0, 4)
mbeq = expr.to_manybody_equation("R")

Here we finally generate the CCSD equations. We use the utility function `generate_equation` to extract the equations corresponding to a given number of creation and annihilation operators and generated Python functions that we then define with the command `exec`

In [24]:
energy_eq = generate_equation(mbeq, 0, 0)
t1_eq = generate_equation(mbeq, 1, 1)
t2_eq = generate_equation(mbeq, 2, 2)

exec(energy_eq)
exec(t1_eq)
exec(t2_eq)

# show what do these functions look like
print(energy_eq)

def evaluate_residual_0_0(H,T):
    # contributions to the residual
    R = 0.0
    R += 1.000000000 * np.einsum("ai,ia->",H["vo"],T["ov"],optimize="optimal")
    R += 0.250000000 * np.einsum("abij,ijab->",H["vvoo"],T["oovv"],optimize="optimal")
    R += 0.500000000 * np.einsum("abij,jb,ia->",H["vvoo"],T["ov"],T["ov"],optimize="optimal")
    return R


## CCSD algorithm

Here we code a simple loop in which we evaluate the energy and residuals of the CCSD equations and update the amplitudes

In [25]:
T = {"ov": np.zeros((nocc, nvir)), "oovv": np.zeros((nocc, nocc, nvir, nvir))}

header = "Iter.     Energy [Eh]    Corr. energy [Eh]       |R|       "
print("-" * len(header))
print(header)
print("-" * len(header))

start = time.perf_counter()

maxiter = 50

for i in range(maxiter):
    # 1. compute energy and residuals
    R = {}
    Ecorr_w = evaluate_residual_0_0(H, T)
    Etot_w = Eref + Ecorr_w
    R["ov"] = evaluate_residual_1_1(H, T)
    Roovv = evaluate_residual_2_2(H, T)
    R["oovv"] = antisymmetrize_residual_2_2(Roovv, nocc, nvir)

    # 2. amplitude update
    update_cc_amplitudes(T, R, invD, 2)

    # 3. check for convergence
    norm_R = np.sqrt(np.linalg.norm(R["ov"]) ** 2 + np.linalg.norm(R["oovv"]) ** 2)
    print(f"{i:3d}    {Etot_w:+.12f}    {Ecorr_w:+.12f}    {norm_R:e}")
    if norm_R < 1.0e-8:
        break

end = time.perf_counter()
t = end - start

print("-" * len(header))
print(f"CCSD total energy                   {Etot_w:+.12f} [Eh]")
print(f"CCSD correlation energy             {Ecorr_w:+.12f} [Eh]")
print(f"Reference CCSD correlation energy   {Ecorr_ref:+.12f} [Eh]")
print(f"Error                               {Ecorr_w - Ecorr_ref:+.12e} [Eh]")
print(f"Timing                              {t:.4f} [s]")
assert np.isclose(Ecorr_w, Ecorr_ref)

-----------------------------------------------------------
Iter.     Energy [Eh]    Corr. energy [Eh]       |R|       
-----------------------------------------------------------
  0    -74.942079898873    +0.000000000000    6.792487e-01
  1    -74.991229543542    -0.049149644668    2.043067e-01
  2    -75.004838116006    -0.062758217133    8.209712e-02
  3    -75.009476493786    -0.067396594913    3.663817e-02
  4    -75.011304448168    -0.069224549294    1.718501e-02
  5    -75.012087669607    -0.070007770733    8.404825e-03
  6    -75.012439954105    -0.070360055231    4.270144e-03
  7    -75.012603732501    -0.070523833627    2.239147e-03
  8    -75.012681944941    -0.070602046068    1.202385e-03
  9    -75.012720205373    -0.070640306500    6.564612e-04
 10    -75.012739341188    -0.070659442314    3.623852e-04
 11    -75.012749106792    -0.070669207918    2.014838e-04
 12    -75.012754180417    -0.070674281544    1.125382e-04
 13    -75.012756857366    -0.070676958492    6.30418

### Build $\bar{H}$

In [26]:
for key in mbeq.keys():
    if key=='|': continue
    func = generate_tensor_block(mbeq[key], key)
    exec(func)

In [27]:
Hbar = {}
Hbar_oooo = evaluate_residual_oooo(H,T)
Hbar['oooo'] = antisymmetrize_residual_2_2_general(Hbar_oooo, 2, 2, 2, 2)
Hbar_ooov = evaluate_residual_ooov(H,T)
Hbar['ooov'] = antisymmetrize_residual_2_2_general(Hbar_ooov, 2, 2, 2, 1)
Hbar_oovv = evaluate_residual_oovv(H,T)
Hbar['oovv'] = antisymmetrize_residual_2_2_general(Hbar_oovv, 2, 2, 2, 2)
Hbar_ovoo = evaluate_residual_ovoo(H,T)
Hbar['ovoo'] = antisymmetrize_residual_2_2_general(Hbar_ovoo, 2, 2, 1, 2)
Hbar_ovov = evaluate_residual_ovov(H,T)
Hbar['ovov'] = antisymmetrize_residual_2_2_general(Hbar_ovov, 2, 2, 1, 1)
Hbar_ovvv = evaluate_residual_ovvv(H,T)
Hbar['ovvv'] = antisymmetrize_residual_2_2_general(Hbar_ovvv, 2, 2, 1, 2)
Hbar['oo'] = evaluate_residual_oo(H,T)
Hbar['ov'] = evaluate_residual_ov(H,T)
Hbar_vvoo = evaluate_residual_vvoo(H,T)
Hbar['vvoo'] = antisymmetrize_residual_2_2_general(Hbar_vvoo, 2, 2, 2, 2)
Hbar_vvov = evaluate_residual_vvov(H,T)
Hbar['vvov'] = antisymmetrize_residual_2_2_general(Hbar_vvov, 2, 2, 2, 1)
Hbar_vvvv = evaluate_residual_vvvv(H,T)
Hbar['vvvv'] = antisymmetrize_residual_2_2_general(Hbar_vvvv, 2, 2, 2, 2)
Hbar['vo'] = evaluate_residual_vo(H,T)
Hbar['vv'] = evaluate_residual_vv(H,T)

### Generate the 'Slater rules' for EOM

In [28]:
sd = ['v+ o','v+ v+ o o']
Hbar_op = Hop
THT = w.op('bra',sd).adjoint() @ Hbar_op @ w.op('c',sd)
expr = wt.contract(THT, 0, 0)
mbeq = expr.to_manybody_equation('sigma')

In [29]:
for eq in mbeq['|']:
    print(eq.compile('einsum'))

sigma += -1.000000000 * np.einsum("ji,aj,ia->",H["oo"],bra["vo"],c["ov"],optimize="optimal")
sigma += -0.500000000 * np.einsum("ji,abjk,ikab->",H["oo"],bra["vvoo"],c["oovv"],optimize="optimal")
sigma += 1.000000000 * np.einsum("ai,bj,ijab->",H["vo"],bra["vo"],c["oovv"],optimize="optimal")
sigma += 0.125000000 * np.einsum("klij,abkl,ijab->",H["oooo"],bra["vvoo"],c["oovv"],optimize="optimal")
sigma += 0.500000000 * np.einsum("kaij,bk,ijab->",H["ovoo"],bra["vo"],c["oovv"],optimize="optimal")
sigma += 0.500000000 * np.einsum("jkia,abjk,ib->",H["ooov"],bra["vvoo"],c["ov"],optimize="optimal")
sigma += -1.000000000 * np.einsum("jbia,aj,ib->",H["ovov"],bra["vo"],c["ov"],optimize="optimal")
sigma += -1.000000000 * np.einsum("jbia,acjk,ikbc->",H["ovov"],bra["vvoo"],c["oovv"],optimize="optimal")
sigma += 0.500000000 * np.einsum("bcia,aj,ijbc->",H["vvov"],bra["vo"],c["oovv"],optimize="optimal")
sigma += 1.000000000 * np.einsum("ia,abij,jb->",H["ov"],bra["vvoo"],c["ov"],optimize="optimal")
sigma +=

In [30]:
sigma = {}
sigma['s'] = np.zeros((nocc, nvir))
sigma['d'] = np.zeros((nocc, nocc, nvir, nvir))
sigma['s'][-1,-1] = 1

In [31]:
for eq in mbeq['|']:
    print(w.compile_sigma_vector(eq))

sigma["vo"] += -1.00000000 * np.einsum('ij,ja->ai', H["oo"],c["ov"],optimize='optimal')
sigma["vvoo"] += -0.50000000 * np.einsum('ij,jkab->abik', H["oo"],c["oovv"],optimize='optimal')
sigma["vo"] += +1.00000000 * np.einsum('ai,ijab->bj', H["vo"],c["oovv"],optimize='optimal')
sigma["vvoo"] += +0.12500000 * np.einsum('ijkl,klab->abij', H["oooo"],c["oovv"],optimize='optimal')
sigma["vo"] += +0.50000000 * np.einsum('iajk,jkab->bi', H["ovoo"],c["oovv"],optimize='optimal')
sigma["vvoo"] += +0.50000000 * np.einsum('ijka,kb->abij', H["ooov"],c["ov"],optimize='optimal')
sigma["vo"] += -1.00000000 * np.einsum('iajb,ja->bi', H["ovov"],c["ov"],optimize='optimal')
sigma["vvoo"] += -1.00000000 * np.einsum('iajb,jkac->bcik', H["ovov"],c["oovv"],optimize='optimal')
sigma["vo"] += +0.50000000 * np.einsum('abic,ijab->cj', H["vvov"],c["oovv"],optimize='optimal')
sigma["vvoo"] += +1.00000000 * np.einsum('ia,jb->abij', H["ov"],c["ov"],optimize='optimal')
sigma["vo"] += +1.00000000 * np.einsum('ab,ia->bi', 

### (Optional) Integral generation
For completeness, we document here how `sr-h2o-cc-pvdz-spinorbital.npy` was generated. You will need to install PySCF to run the following cell. Due to the size, we do not provide the cc-pVTZ integrals in the repository, but you can generate it with the cell below and see the difference to the spin-integrated code in the other notebook.

In [67]:
import pyscf, pyscf.cc, pyscf.mp

mol = pyscf.gto.M(atom="""
O 
H 1 1.1
H 1 1.1 2 104
""", basis='sto-3g')

mf = pyscf.scf.RHF(mol)
_ = mf.kernel()
cc = pyscf.cc.CCSD(mf)
_ = cc.kernel()
mp = pyscf.mp.MP2(mf)
_ = mp.kernel()

nocc = mol.nelectron
nvir = (mol.nao*2 - nocc)

eri = pyscf.ao2mo.full(mol.intor('int2e'), mf.mo_coeff)
eri = eri.swapaxes(1, 2)

blocks = get_index_blocks(energy_eq+t1_eq+t2_eq)

V = np.zeros((mol.nao*2, mol.nao*2, mol.nao*2, mol.nao*2))
V[::2,::2,::2,::2] = V[1::2,1::2,1::2,1::2] = eri - eri.swapaxes(2,3) # <aa||aa> and <bb||bb>
V[::2,1::2,::2,1::2] = V[1::2,::2,1::2,::2] = eri # <ab||ab> = <ba||ba> = <ab|ab>
V[::2,1::2,1::2,::2] = V[1::2,::2,::2,1::2] = -eri.swapaxes(2,3) # <ab||ba> = <ba||ab> = -<ab|ab>
F_spatorb = np.diag(mf.mo_energy)
F = np.zeros((mol.nao*2, mol.nao*2))
F[::2,::2] = F[1::2,1::2] = F_spatorb
sl = {'o': slice(0, nocc), 'v': slice(nocc, nocc+nvir)}
H = {}
for block in blocks:
    if len(block) == 2:
        H[block] = F[sl[block[0]], sl[block[1]]]
    else:
        H[block] = V[sl[block[0]], sl[block[1]], sl[block[2]], sl[block[3]]]

with open('sr-h2o-sto-3g-spinorbital.npy', 'wb') as f:
    np.save(f, mf.e_tot)
    np.save(f, cc.e_corr)
    np.save(f, (nocc,nvir))
    np.save(f, H, allow_pickle=True)

converged SCF energy = -74.9420798988735
E(CCSD) = -75.01275999574094  E_corr = -0.07068009686744721
E(MP2) = -74.9912295435419  E_corr = -0.0491496446683854
E(SCS-MP2) = -74.9983674136603  E_corr = -0.0562875147868231


In [41]:
import logging
import warnings

import numpy as np
import numpy.linalg
import scipy
import scipy.linalg
import scipy.sparse
import scipy.sparse.linalg


class Davidson(object):
    """Davidson-Liu algorithm to get the n states with smallest eigenvalues."""

    def __init__(self, matrix, max_subspace=100, max_iterations=300, eps=1e-6, build_sigma_vector=None):
        self.matrix = matrix
        self.diagonal = matrix.diagonal()
        self.max_subspace = max_subspace
        self.max_iterations = max_iterations
        self.eps = eps

        self.build_sigma_vector = build_sigma_vector
        if build_sigma_vector is None:
            self.build_sigma_vector = lambda mat, v: np.einsum('nm, m->n', mat, v, optimize='optimal')
        

    def kernel(self, n_lowest=1, initial_guess=None):
        # Checks for number of states desired, should be in the range of
        # [0, max_subspace).
        if n_lowest <= 0 or n_lowest >= self.max_subspace:
            raise ValueError(
                'n_lowest {} is supposed to be in [1, {}).'.format(
                    n_lowest, self.max_subspace
                )
            )

        # Checks for the initial guess.
        if initial_guess is None:
            initial_guess = np.zeros((len(self.diagonal), n_lowest))
            np.fill_diagonal(initial_guess, 1)

        sucess = False
        niter = 0
        guess_v = initial_guess

        while niter < self.max_iterations and not sucess:
            guess_mv = np.zeros((self.matrix.shape[0], guess_v.shape[1]))
            for i in range(guess_v.shape[1]):
                guess_mv[:,i] = self.build_sigma_vector(self.matrix, guess_v[:,i])
            # replace with sigma vector build
            
            guess_vmv = np.einsum('ni,nj->ij', guess_v,
                                  guess_mv, optimize='optimal')
            trial_lambda, trial_transformation = np.linalg.eigh(guess_vmv)

            # 1. Sorts eigenvalues in ascending order.
            # sorted_index = list(reversed(trial_lambda.argsort()[::-1]))
            # trial_lambda = trial_lambda[sorted_index]
            # trial_transformation = trial_transformation[:, sorted_index]

            if len(trial_lambda) > n_lowest:
                trial_lambda = trial_lambda[:n_lowest]
                trial_transformation = trial_transformation[:, :n_lowest]

            # 2. Estimates errors based on diagonalization in the smaller space.
            # Guess eigenvectors in the original space.
            trial_v = np.dot(guess_v, trial_transformation)
            # Guess Ax in the original space.
            trial_mv = np.zeros((self.matrix.shape[0], trial_v.shape[1]))
            for i in range(trial_v.shape[1]):
                trial_mv[:,i] = self.build_sigma_vector(self.matrix, trial_v[:,i])
            # Residual vectors in the original space.
            trial_error = trial_mv - trial_v * trial_lambda

            # 3. Gets new directions from error vectors.
            max_error = 0  # Maximum error for the current iteration.
            new_directions = []
            full_dim = trial_v.shape[0]
            for i in range(n_lowest):
                current_error_v = trial_error[:, i]
                if np.max(np.abs(current_error_v)) < self.eps:
                    continue
                max_error = max(max_error, np.linalg.norm(current_error_v))

                new_direction = []
                M = np.ones(full_dim)
                for j in range(full_dim):
                    diff = self.diagonal[j] - trial_lambda[i]
                    if numpy.abs(diff) > self.eps:
                        M[j] /= diff
                    else:
                        M[j] /= self.eps

                numerator = M * current_error_v
                denominator = M * trial_v[:, i]
                # This line is fine.
                new_direction = -current_error_v + \
                    (trial_v[:, i] * numpy.dot(trial_v[:, i],
                     numerator) / numpy.dot(trial_v[:, i], denominator))
                # new_direction *= M

                # new_direction = M * current_error_v # This is the Davidson-Liu way.

                # P = np.identity(full_dim) - np.outer(trial_v[:, i], trial_v[:, i])
                # A_tilde = np.diagonal(self.matrix) - np.identity(full_dim) * trial_lambda[i]
                # M_tilde = np.einsum('ij, jk, kl->il', P, A_tilde, P, optimize='optimal')
                # new_direction, _ = scipy.sparse.linalg.cg(M_tilde, -current_error_v, atol = self.eps, maxiter = self.max_iterations) # This is the Jacobi-Davidson way.

                new_directions.append(new_direction)

            if new_directions:
                # stack new_directions along the axis 0, then transpose, finally hstack.
                guess_v = np.hstack([guess_v, np.stack(new_directions).T])

            print(
                f"Eigenvalues for iteration {niter}: {trial_lambda}, error is {max_error}."
            )

            if max_error < self.eps:
                success = True
                break

            # 4. Deals with new directions to make sure they're orthonormal.
            ortho_num = guess_mv.shape[1]  # Already orthonormal
            for i in range(ortho_num, guess_v.shape[1]):
                vec_i = guess_v[:, i]
                for j in range(i):
                    vec_i -= guess_v[:, j] * np.dot(guess_v[:, j], vec_i)

                # Makes sure the new vector is not too small.
                if np.linalg.norm(vec_i) < self.eps:
                    continue

                guess_v[:, i] = vec_i / np.linalg.norm(vec_i)
                ortho_num += 1

            # 5. Limits the size of the subspace.
            if guess_v.shape[1] > self.max_subspace:
                print("Collapsing the subspace.")
                guess_v = trial_v
                guess_mv = trial_mv

            niter += 1

        return trial_lambda, trial_v


if (__name__ == '__main__'):
    np.random.seed(100)
    A = np.random.rand(500, 500)
    A = A + A.T

    davidson = Davidson(A)
    trial_lambda, trial_v = davidson.kernel(n_lowest=2)
    print(f"Eigenvalues from Davidson: {trial_lambda}")
    evals, evecs = np.linalg.eigh(A)
    print(f"Eigenvalues from Numpy: {evals[:2]}")


Eigenvalues for iteration 0: [-0.21008765  1.61915173], error is 32.299435853061595.
Eigenvalues for iteration 1: [-8.87674753 -0.83002587], error is 9.300414612916818.
Eigenvalues for iteration 2: [-12.48270604  -8.89702452], error is 6.080874449312598.
Eigenvalues for iteration 3: [-14.41449853 -12.80648843], error is 5.056861433829724.
Eigenvalues for iteration 4: [-15.40025146 -15.02022691], error is 3.3857410208173007.
Eigenvalues for iteration 5: [-16.15138964 -15.86407193], error is 2.352132350316581.
Eigenvalues for iteration 6: [-16.62924502 -16.21225627], error is 1.7713494674192907.
Eigenvalues for iteration 7: [-16.92820487 -16.49688728], error is 1.5543230637741108.
Eigenvalues for iteration 8: [-17.17605032 -16.79405231], error is 1.5976792698606002.
Eigenvalues for iteration 9: [-17.39317603 -17.02780442], error is 1.338531626371993.
Eigenvalues for iteration 10: [-17.56703422 -17.19961644], error is 1.1362171553141123.
Eigenvalues for iteration 11: [-17.68075609 -17.289